In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime

# 1. 셀레니움 드라이버 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(), options=options)

# 2. URL 접속
url = "https://www.yes24.com/Product/Category/BestSeller?CategoryNumber=001&sumgb=06"
driver.get(url)

# 3. 데이터 저장 리스트
books_data = []

# 4. 전체 페이지 순회
for page in range(1, 43):
    print(f"크롤링 중: {page}페이지")

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul#yesBestList > li"))
        )
    except Exception as e:
        print(f"페이지 로딩 실패: {e}")
        break

    books = driver.find_elements(By.CSS_SELECTOR, "ul#yesBestList > li")

    for book in books:
        try:
            rank = book.find_element(By.CSS_SELECTOR, "em.ico.rank").text.strip()
        except:
            rank = ""

        try:
            title = book.find_element(By.CSS_SELECTOR, "a.gd_name").text.strip()
        except:
            title = ""

        try:
            authors = book.find_elements(By.CSS_SELECTOR, "span.authPub.info_auth a")
            author = ", ".join(a.text.strip() for a in authors)
        except:
            author = ""

        try:
            publishers = book.find_elements(By.CSS_SELECTOR, "span.authPub.info_pub a")
            publisher = ", ".join(p.text.strip() for p in publishers)
        except:
            publisher = ""

        try:
            price = book.find_element(By.CSS_SELECTOR, "strong.txt_num em").text.strip()
        except:
            price = ""

        books_data.append({
            "순위": rank,
            "제목": title,
            "저자": author,
            "출판사": publisher,
            "가격": price
        })

    # 다음 페이지로 이동
    if page < 42:
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[onclick*='changeBestSellerParam(this, {page+1})']"))
            )
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(1.5)
        except Exception as e:
            print(f"페이지 이동 실패: {e}")
            break

# 5. 결과 저장
driver.quit()

# 중복 제거 (제목 기준)
df = pd.DataFrame(books_data).drop_duplicates(subset="제목")

# 날짜 포함된 파일명 저장
today_str = datetime.now().strftime("%Y%m%d")
filename = f"yes24_bestseller_{today_str}.csv"
df.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"✅ 전체 크롤링 완료 ({len(df)}권) 및 CSV 저장: {filename}")
df

크롤링 중: 1페이지
크롤링 중: 2페이지
크롤링 중: 3페이지
크롤링 중: 4페이지
크롤링 중: 5페이지
크롤링 중: 6페이지
크롤링 중: 7페이지
크롤링 중: 8페이지
크롤링 중: 9페이지
크롤링 중: 10페이지
크롤링 중: 11페이지
크롤링 중: 12페이지
크롤링 중: 13페이지
크롤링 중: 14페이지
크롤링 중: 15페이지
크롤링 중: 16페이지
크롤링 중: 17페이지
크롤링 중: 18페이지
크롤링 중: 19페이지
크롤링 중: 20페이지
크롤링 중: 21페이지
크롤링 중: 22페이지
크롤링 중: 23페이지
크롤링 중: 24페이지
크롤링 중: 25페이지
크롤링 중: 26페이지
크롤링 중: 27페이지
크롤링 중: 28페이지
크롤링 중: 29페이지
크롤링 중: 30페이지
크롤링 중: 31페이지
크롤링 중: 32페이지
크롤링 중: 33페이지
크롤링 중: 34페이지
크롤링 중: 35페이지
크롤링 중: 36페이지
크롤링 중: 37페이지
크롤링 중: 38페이지
크롤링 중: 39페이지
크롤링 중: 40페이지
크롤링 중: 41페이지
크롤링 중: 42페이지
✅ 전체 크롤링 완료 (947권) 및 CSV 저장: yes24_bestseller_20250618.csv


,순위,제목,저자,출판사,가격
0,1,유지만 다이어트 레시피,유지만,길벗,"19,800"
1,2,"첫 여름, 완주",김금희,무제,"15,300"
2,3,혼모노,성해나,창비,"16,200"
3,4,청춘의 독서,유시민,웅진지식하우스,"17,010"
4,5,우리의 낙원에서 만나자,하태완,북로망스,"17,550"
...,...,...,...,...,...
993,994,매직 스플릿,박성현,경이로움,"15,300"
994,995,만화 나 혼자만 레벨업 12,"추공, 장성락(REDICE STUDIO)",디앤씨웹툰비즈,"14,400"
995,996,우연은 비켜 가지 않는다,"줄리언 반스, 정영목",다산책방,"15,750"
996,997,큰별쌤과 재미있게 공부하는 초등 한국사능력검정시험,최태성,이투스북,"17,100"
